クックブック: Langchain 統合
https://langfuse.com/docs/integrations/langchain/example-python

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langfuse.callback import CallbackHandler
 
langfuse_handler = CallbackHandler()

In [6]:
# Tests the SDK connection with the server
langfuse_handler.auth_check()

True

### Langchain 式言語 (LCEL) におけるシーケンシャル チェーン

In [7]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
 
langfuse_handler = CallbackHandler(
    trace_name="langchain-example-20240825-001",
    session_id="user-1234",
    user_id="session-1234",
)
 
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
chain1 = prompt1 | model | StrOutputParser()
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)
 
chain2.invoke({"person": "obama", "language": "japanese"}, config={"callbacks":[langfuse_handler]})

'バラク・オバマがしばしば関連付けられる都市は、アメリカ合衆国のイリノイ州シカゴです。彼は1980年代にそこに移り住み、州上院議員として、そして後にアメリカ合衆国上院議員として政治キャリアを始め、最終的には大統領になりました。'

In [8]:
# Async Invoke
await chain2.ainvoke({"person": "biden", "language": "german"}, config={"callbacks":[langfuse_handler]})
  

'Die Stadt Wilmington, in der Joe Biden lebt, befindet sich in den Vereinigten Staaten von Amerika.'

In [10]:
# Batch
chain2.batch([{"person": "elon musk", "language": "german"}, {"person": "mark zuckerberg", "language": "japanese"}], config={"callbacks":[langfuse_handler]})
 

['Die Stadt, in der Elon Musk geboren wurde, Pretoria, liegt in Südafrika.',
 'マーク・ザッカーバーグはアメリカ合衆国のニューヨーク州のホワイトプレインズ出身です。彼は近くのダブズ・フェリーで育ちました。']

In [11]:
# Async Batch
await chain2.abatch([{"person": "jeff bezos", "language": "german"}, {"person": "tim cook", "language": "japanese"}], config={"callbacks":[langfuse_handler]})
 

['Die Stadt Albuquerque, New Mexico, befindet sich in den Vereinigten Staaten.',
 'ティム・クックはアメリカ合衆国のモバイル、アラバマ出身です。']

In [12]:
# Stream
for chunk in chain2.stream({"person": "steve jobs", "language": "english"}, config={"callbacks":[langfuse_handler]}):
    print("Streaming chunk:", chunk)

Streaming chunk: 
Streaming chunk: Steve
Streaming chunk:  Jobs
Streaming chunk:  was
Streaming chunk:  born
Streaming chunk:  in
Streaming chunk:  the
Streaming chunk:  United
Streaming chunk:  States
Streaming chunk: ,
Streaming chunk:  specifically
Streaming chunk:  in
Streaming chunk:  San
Streaming chunk:  Francisco
Streaming chunk: ,
Streaming chunk:  California
Streaming chunk: .
Streaming chunk:  He
Streaming chunk:  grew
Streaming chunk:  up
Streaming chunk:  in
Streaming chunk:  Cupertino
Streaming chunk: ,
Streaming chunk:  California
Streaming chunk: ,
Streaming chunk:  which
Streaming chunk:  is
Streaming chunk:  also
Streaming chunk:  in
Streaming chunk:  the
Streaming chunk:  United
Streaming chunk:  States
Streaming chunk: .
Streaming chunk: 


In [13]:
# Async Stream
async for chunk in chain2.astream({"person": "bill gates", "language": "english"}, config={"callbacks":[langfuse_handler]}):
    print("Async Streaming chunk:", chunk)

Async Streaming chunk: 
Async Streaming chunk: Seattle
Async Streaming chunk: ,
Async Streaming chunk:  Washington
Async Streaming chunk: ,
Async Streaming chunk:  is
Async Streaming chunk:  in
Async Streaming chunk:  the
Async Streaming chunk:  United
Async Streaming chunk:  States
Async Streaming chunk: .
Async Streaming chunk: 


### 会話チェーン

In [8]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
 
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
 
conversation = ConversationChain(
    llm=llm, memory=ConversationBufferMemory()
)

In [9]:
# Create a callback handler with a session
langfuse_handler = CallbackHandler(session_id="conversation_chain")

conversation.predict(input="Hi there!", callbacks=[langfuse_handler])
conversation.predict(input="How to build great developer tools?", callbacks=[langfuse_handler])
conversation.predict(input="Summarize your last response", callbacks=[langfuse_handler])

'Sure! To build great developer tools, focus on understanding your audience and their needs, clearly define the problem your tool solves, and prioritize user-centric design for a good UI/UX. Ensure robust functionality, performance, and reliability, and provide comprehensive documentation and support. Gather user feedback for continuous improvement, promote your tool within developer communities, and consider open-sourcing it for collaboration. Lastly, stay updated on tech trends to keep your tool relevant. Do you have a specific type of tool in mind?'

### 検索QA

In [10]:
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
 
langfuse_handler = CallbackHandler()
 
urls = [
    "https://raw.githubusercontent.com/langfuse/langfuse-docs/main/public/state_of_the_union.txt",
]
loader = SeleniumURLLoader(urls=urls)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
query = "What did the president say about Ketanji Brown Jackson"
chain = RetrievalQA.from_chain_type(
    llm,
    retriever=docsearch.as_retriever(search_kwargs={"k": 1}),
)
 
chain.invoke(query, config={"callbacks":[langfuse_handler]})

{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': "The president described Ketanji Brown Jackson as one of the nation's top legal minds and stated that she will continue Justice Breyer's legacy of excellence."}

### エージェント

### AzureオープンAI

### シーケンシャルチェーン [レガシー]

## トレースにスコアを追加する

In [12]:
from langfuse.decorators import langfuse_context, observe
 
@observe(as_type="generation")
def deeply_nested_llm_call():
    # Enrich the current observation with a custom name, input, and output
    langfuse_context.update_current_observation(
        name="Deeply nested LLM call222", input="Ping?", output="Pong!"
    )
    # Set the parent trace's name from within a nested observation
    langfuse_context.update_current_trace(
        name="Trace name set from deeply_nested_llm_call",
        session_id="1234",
        user_id="5678",
        tags=["tag1", "tag2"],
        public=True
    )
 
@observe()
def nested_span():
    # Update the current span with a custom name and level
    langfuse_context.update_current_observation(name="Nested Span", level="WARNING")
    deeply_nested_llm_call()
 
@observe()
def main():
    nested_span()
 
# Execute the main function to generate the enriched trace
main()

### Langfuse Python SDK との相互運用性

In [13]:
from langfuse.decorators import langfuse_context, observe
 
# Create a trace via Langfuse decorators and get a Langchain Callback handler for it
@observe() # automtically log function as a trace to Langfuse
def main():
    # update trace attributes (e.g, name, session_id, user_id)
    langfuse_context.update_current_trace(
        name="custom-trace",
        session_id="user-1234",
        user_id="session-1234",
    )
    # get the langchain handler for the current trace
    langfuse_context.get_current_langchain_handler()
 
    # use the handler to trace langchain runs ...
 
main()

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
 
prompt = ChatPromptTemplate.from_template("what is the city {person} is from?")
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
 
chain = prompt | model

In [15]:
from langfuse.decorators import langfuse_context, observe
 
# On span "Physics"."Favorites"
@observe()  # decorator to automatically log function as sub-span to Langfuse
def favorites():
    # get the langchain handler for the current sub-span
    langfuse_handler = langfuse_context.get_current_langchain_handler()
    # invoke chain with langfuse handler
    chain.invoke({"person": "Richard Feynman"},
                 config={"callbacks": [langfuse_handler]})
 
# On span "Physics"
@observe()  # decorator to automatically log function as span to Langfuse
def physics():
    # get the langchain handler for the current span
    langfuse_handler = langfuse_context.get_current_langchain_handler()
    # invoke chains with langfuse handler
    chain.invoke({"person": "Albert Einstein"},
                 config={"callbacks": [langfuse_handler]})
    chain.invoke({"person": "Isaac Newton"},
                 config={"callbacks": [langfuse_handler]})
    favorites()
 
# On trace
@observe()  # decorator to automatically log function as trace to Langfuse
def main():
    # get the langchain handler for the current trace
    langfuse_handler = langfuse_context.get_current_langchain_handler()
    # invoke chain with langfuse handler
    chain.invoke({"person": "Alan Turing"},
                 config={"callbacks": [langfuse_handler]})
    physics()
 
main()

In [16]:
from langfuse.decorators import langfuse_context, observe
 
# On span "Physics"."Favorites"
# @observe()  # decorator to automatically log function as sub-span to Langfuse
def favorites():
    # get the langchain handler for the current sub-span
    langfuse_handler = langfuse_context.get_current_langchain_handler()
    # invoke chain with langfuse handler
    chain.invoke({"person": "Richard Feynman"},
                 config={"callbacks": [langfuse_handler]})
 
# On span "Physics"
# @observe()  # decorator to automatically log function as span to Langfuse
def physics():
    # get the langchain handler for the current span
    langfuse_handler = langfuse_context.get_current_langchain_handler()
    # invoke chains with langfuse handler
    chain.invoke({"person": "Albert Einstein"},
                 config={"callbacks": [langfuse_handler]})
    chain.invoke({"person": "Isaac Newton"},
                 config={"callbacks": [langfuse_handler]})
    favorites()
 
# On trace
@observe()  # decorator to automatically log function as trace to Langfuse
def main():
    # get the langchain handler for the current trace
    langfuse_handler = langfuse_context.get_current_langchain_handler()
    # invoke chain with langfuse handler
    chain.invoke({"person": "Alan Turing"},
                 config={"callbacks": [langfuse_handler]})
    physics()
 
main()